# 00_ETL_modelo

## 01_import_bibliotecas

In [1]:
import pandas as pd
import json
from pandas import json_normalize
import requests

## 02_import_dados

In [2]:
# import de dados da decision

vagas = "https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/01_fontes/arquivos_decision/vagas.json"
prospects = "https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/01_fontes/arquivos_decision/prospects.json"
applicants = "https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/01_fontes/arquivos_decision/applicants.json"

In [3]:
# lê os dados do github

# vagas
response_vagas = requests.get(vagas)
data_vagas = response_vagas.json()

# prospects
response_prospects = requests.get(prospects)
data_prospects = response_prospects.json()

# Aplicants
response_applicants = requests.get(applicants)
data_applicants = response_applicants.json()

## 03_tratamento_json

In [4]:
# Trata Vagas

records_vagas = [dict(id_vaga=k, **v) for k, v in data_vagas.items()]

df_vagas = pd.json_normalize(records_vagas, sep="_")

# Remove os prefixos das colunas
prefixos_para_remover_vagas = [
    'informacoes_basicas_',
    'perfil_vaga_',
    'beneficios_'
]

for prefixo in prefixos_para_remover_vagas:
    df_vagas.columns = df_vagas.columns.str.replace(f'^{prefixo}', '', regex=True)

In [5]:
# Trata Prospects

records_prospects = [dict(id_vaga=k, **v) for k, v in data_prospects.items()]

df_prospects = pd.json_normalize(
    records_prospects,
    record_path='prospects',
    meta=['id_vaga', 'titulo', 'modalidade'],
    sep='_'
)

# Remove os prefixos das colunas
prefixos_para_remover_prospects = [
'prospects_',  
]

for prefixo in prefixos_para_remover_prospects:
    df_prospects.columns = df_prospects.columns.str.replace(f'^{prefixo}', '', regex=True)
    
df_prospects.rename(columns={'codigo': 'id_candidato'}, inplace=True)

In [6]:
# Trata Aplicants

records_applicants = [dict(id_candidato=k, **v) for k, v in data_applicants.items()]

df_applicants = pd.json_normalize(records_applicants, sep="_")

# Remove os prefixos das colunas
prefixos_para_remover_applicants = [
    'infos_basicas_',
    'informacoes_pessoais_',
    'informacoes_profissionais_',
    'formacao_e_idiomas_',
    'cargo_atual_'
]

for prefixo in prefixos_para_remover_applicants:
    df_applicants.columns = df_applicants.columns.str.replace(f'^{prefixo}', '', regex=True)

## 04_juntando_arquivos

In [7]:
# prospects + applicants
df_inter = df_prospects.merge(df_applicants, on="id_candidato", how="left", suffixes=('_tabelaprospect', '_tabelaapplicant'))

In [8]:
# dados + Vagas
df_geral = df_inter.merge(df_vagas, on="id_vaga", how="left", suffixes=('', '_tabelavaga'))

## 05_Salvando_base_unificada_no_git

In [ ]:
Caminho = "C:\\Users\\pedro\\Documents\\Área de Trabalho\\tech_challenge_5\\01_fontes\\arquivos_decision\\fontes_tratadas\\"
arquivo = "base_unificada.csv"

In [ ]:
df_geral.to_csv(Caminho + arquivo, encoding ="utf8" , sep=";" ,index=False)

## teste

In [9]:
Caminho = "C:\\Users\\pedro\\Documents\\Área de Trabalho\\tech_challenge_5\\01_fontes\\arquivos_decision\\fontes_tratadas\\"

In [ ]:
import re

In [ ]:
# df_candidatos = df_applicants[
#     ['area_atuacao',
#     'certificacoes',
#     'data_atualizacao',
#     'data_criacao',
#     'nivel_academico',
#     'nivel_espanhol',
#     'nivel_ingles',
#     'outras_certificacoes',
#     'outro_idioma']
# ]

# df_candidatos = df_applicants.loc[:, ~df_applicants.T.duplicated()]

In [12]:
df_candidatos = df_applicants[
    ['ano_conclusao',
'area_atuacao',
'cargo_atual',
'certificacoes',
'conhecimentos_tecnicos',
'cursos',
'cv_en',
'cv_pt',
'data_admissao',
'data_atualizacao',
'data_criacao',
'data_ultima_promocao',
'experiencias',
'fonte_indicacao',
'id_candidato',
'nivel_academico',
'nivel_espanhol',
'nivel_ingles',
'nivel_profissional',
'nome',
'objetivo_profissional',
'outras_certificacoes',
'outro_curso',
'outro_idioma',
'projeto_atual',
'qualificacoes',
'remuneracao',
'sabendo_de_nos_por',
]
]

In [15]:
df_candidatos.to_csv(Caminho + "01_candidatos.csv", encoding ="utf8" , sep="," ,index=False)

In [14]:
# Salvar para Excel agora deve funcionar
df_candidatos.to_excel(Caminho + "01_candidatos.xlsx", index=False, sheet_name="Candidatos", engine='openpyxl')

IllegalCharacterError: Profissional de tecnologia com 30 anos de experiência, trabalhando há 11 anos com ERP SAP ECC e atualmente com S4/Hana (área funcional/ABAP) em companhias de grande e médio porte, atuando em projetos críticos com competências de gestão do módulo MM (gestão de materiais) e Localização Brasil; Consultor na implantação do SAP S4/HANA com Fiori; Academia Oficial SAP (PUR, IM, IV, MRP, WM, CBP) concluída em 2010 e Localização Brasil (TAXBRA) concluída em 2011; Consultoria na implantação do S4/HANA. Domínio dos cenários de negócios, atuando no mapeamento de processos, levantamento de requisitos, arquitetura de soluções, parametrização do ERP, condução de testes unitários e integrados, produção de documentação, treinamento de key users e auditoria (hands-on); Gestão de Projetos de tecnologia (SAP/JDE), utilizando metodologia ágil SCRUM e métricas PMBOK aliado às metodologias SAP RoadMap e Activate. Perfil com habilidade de liderança, focado na condução de soluções, comp

In [ ]:
df_candidatos.to_parquet(Caminho + "01_candidatos.parquet", index=False)

In [13]:
import re

# Função para remover caracteres ilegais invisíveis (ex: \x00, \x01, etc.)
def limpar_valor(valor):
    if isinstance(valor, str):
        # Remove caracteres de controle ASCII 0–31
        return re.sub(r'[\x00-\x1F]', '', valor)
    return valor

# Aplica a limpeza a todas as células do DataFrame
df_candidatos = df_candidatos.applymap(limpar_valor)

# Salvar para Excel agora deve funcionar
df_candidatos.to_excel(Caminho + "01_candidatos.xlsx", index=False, sheet_name="Candidatos", engine='openpyxl')

KeyboardInterrupt: 